In [ ]:
import os
import base64
import json
from pathlib import Path
from typing import List, Dict, Union
from html.parser import HTMLParser
from html import escape, unescape

In [ ]:
class ImageEmbedder(HTMLParser):
    def __init__(self, html_path: Path):
        super().__init__()
        self.html_path = html_path
        self.result = ""
        self.success = []
        self.fail = []

    def handle_starttag(self, tag, attrs):
        if tag.lower() == "img":
            attrs_dict = dict(attrs)
            src = attrs_dict.get("src", "")
            img_path = (self.html_path.parent / src).resolve()
            if img_path.exists():
                try:
                    with open(img_path, "rb") as img_file:
                        encoded = base64.b64encode(img_file.read()).decode('utf-8')
                        mime_type = f"image/{img_path.suffix[1:]}"
                        new_src = f"data:{mime_type};base64,{encoded}"
                        attrs_dict["src"] = new_src
                        self.success.append(str(img_path))
                except Exception:
                    self.fail.append(str(img_path))
            else:
                self.fail.append(str(img_path))

            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs_dict.items())
            self.result += f"<img {attrs_str}>"
        else:
            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs)
            self.result += f"<{tag} {attrs_str}>" if attrs_str else f"<{tag}>"

    def handle_endtag(self, tag):
        self.result += f"</{tag}>"

    def handle_data(self, data):
        self.result += data

    def handle_entityref(self, name):
        self.result += f"&{name};"

    def handle_charref(self, name):
        self.result += f"&#{name};"


In [ ]:
class HTMLBatchProcessor:
    def __init__(self, input_paths: List[Union[str, Path]], output_dir: Union[str, Path]):
        self.input_paths = [Path(p) for p in input_paths]
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.results = {
            "success": {},
            "fail": {}
        }

    def get_html_files(self) -> List[Path]:
        html_files = []
        for path in self.input_paths:
            if path.is_file() and path.suffix == ".html":
                html_files.append(path)
            elif path.is_dir():
                html_files.extend(path.rglob("*.html"))  #Explora subdirectorios también
        return html_files

    def process(self) -> Dict[str, Dict[str, List[str]]]:
        html_files = self.get_html_files()
        print(f"Se encontraron {len(html_files)} archivos HTML")

        for html_file in html_files:
            with open(html_file, "r", encoding="utf-8") as f:
                content = f.read()

            parser = ImageEmbedder(html_file)
            parser.feed(content)

            # Guardar nuevo archivo
            output_file = self.output_dir / html_file.name
            with open(output_file, "w", encoding="utf-8") as f_out:
                f_out.write(parser.result)

            self.results["success"][str(html_file)] = parser.success
            self.results["fail"][str(html_file)] = parser.fail

        return self.results

In [ ]:

# Ejecución
if __name__ == "__main__":
    rutas = ["/content/drive/MyDrive/Prueba_tecnica_tuya/htmls_prueba"]
    salida = "/content/drive/MyDrive/Prueba_tecnica_tuya/html_modificados"

    processor = HTMLBatchProcessor(rutas, salida)
    resultado = processor.process()

    total_ok = sum(len(v) for v in resultado["success"].values())
    total_fail = sum(len(v) for v in resultado["fail"].values())
    print(f"\n Imágenes procesadas correctamente: {total_ok}")
    print(f"❌ Imágenes con error: {total_fail}")

    resumen_path = "/content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json"
    with open(resumen_path, "w", encoding="utf-8") as f:
        json.dump(resultado, f, indent=4, ensure_ascii=False)
    print(f"\n Resumen guardado en: {resumen_path}")

Se encontraron 3 archivos HTML

 Imágenes procesadas correctamente: 9
❌ Imágenes con error: 0

 Resumen guardado en: /content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json


In [ ]:
readme_content = """# Prueba Técnica - Tuya

Este repositorio contiene la solución al primer punto de la prueba técnica de Tuya, donde se procesan archivos HTML para reemplazar imágenes por versiones embebidas en formato base64.

## Objetivo

Automatizar el procesamiento de archivos HTML para:

- Buscar imágenes dentro del código HTML.
- Convertirlas a base64.
- Reemplazarlas directamente en el HTML.
- Guardar el HTML modificado y un resumen en formato JSON con las imágenes procesadas correctamente o con error.

## Tecnologías usadas

- Python 3
- Librerías estándar: `os`, `base64`, `json`, `pathlib`, `html.parser`

## Estructura del proyecto

- `htmls_prueba/`: Carpeta de entrada con los archivos HTML originales.
- `html_modificados/`: Carpeta de salida con los archivos HTML modificados.
- `resumen_resultado.json`: Archivo con el resumen del procesamiento.

## Cómo ejecutar

1. Asegúrate de tener tus archivos HTML en la ruta especificada.
2. Ejecuta el script en Google Colab o un entorno local de Python.
3. Verifica el contenido de la carpeta de salida y el archivo `resumen_resultado.json`.

"""

# Ruta donde guardar el README
ruta_readme = "/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/README.md"

# Guardar el archivo
with open(ruta_readme, "w", encoding="utf-8") as f:
    f.write(readme_content)

print(f"README.md creado en: {ruta_readme}")



README.md creado en: /content/drive/MyDrive/Prueba_tecnica_tuya/README.md


In [ ]:
gitignore_content = """
# Ignorar cachés de Python
__pycache__/
*.py[cod]

# Ignorar checkpoints de Jupyter
.ipynb_checkpoints/

# Ignorar archivos temporales
*.swp
*.tmp

# Ignorar archivos ocultos del sistema
.DS_Store
Thumbs.db
"""

ruta_gitignore = "/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/.gitignore"

with open(ruta_gitignore, "w", encoding="utf-8") as f:
    f.write(gitignore_content)

print(f".gitignore creado en: {ruta_gitignore}")

.gitignore creado en: /content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imágenes/.gitignore


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Prueba_tecnica_tuya/ Ejercicio_1_ Procesamiento_Archivos_HTML_con_Imagenes'
print(os.listdir(path))  # Verifica el contenido

['Tuya_P1.ipynb', '.gitignore']


In [2]:
# Cambiar de directorio a la carpeta del proyecto
%cd /content/drive/MyDrive/Prueba_tecnica_tuya/Ejercicio_1_HTML

/content/drive/MyDrive/Prueba_tecnica_tuya/Ejercicio_1_HTML


In [16]:
!git remote remove origin

In [10]:
# Inicializar Git
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/Prueba_tecnica_tuya/Ejercicio_1_HTML/.git/


In [17]:
# Configurar tu nombre de usuario y correo (solo la primera vez)
!git config --global user.email "danieladoncell@gmail.com"
!git config --global user.name "ddoncell94"

In [18]:
# Agregar todos los archivos a Git
!git add .

In [19]:
# Hacer un commit con mensaje
!git commit -m "Primera subida del ejercicio 1: procesamiento de HTML con imágenes en base64"

[master b67d3e7] Primera subida del ejercicio 1: procesamiento de HTML con imágenes en base64
 1 file changed, 1 insertion(+), 1 deletion(-)


In [20]:
# Conectar con el repositorio remoto
!git remote add origin https://github.com/ddoncell94/tuya-technical-assessment.git

In [21]:
# Subir al repositorio remoto
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address
